<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/Mortgage_Payoff_Calculator_(Investor_Cash_Flow_Option).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mortgage Payoff Calculator (Investor Cash Flow Option)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 29th May 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [How to Use Python to Model Paying Off Your Mortgage Early](https://datadrivenmoney.com/how-to-use-python-to-model-paying-off-your-mortgage-early/)
- [Simple Mortgage Calculator With Python and Excel](https://medium.com/swlh/simple-mortgage-calculator-with-python-and-excel-b98dede36720)
- [Mortgage Payoff Calculator](https://www.calculator.net/mortgage-payoff-calculator.html?cloanamount=400000&cloanterm=30&cinterestrate=4&cremainingyear=30&cremainingmonth=0&cadditionalmonth=500&cadditionalyear=0&cadditionalonetime=0&cpayoffoption=original&type=1&x=41&y=17#loanterm)

## <font color="blue">Install Packages</font>

In [2]:
!pip install numpy-financial -q

## <font color="blue">Imports</font>

In [3]:
import pandas as pd
import numpy_financial as npf
from datetime import date, datetime
import plotly.express as px

## <font color="blue">Functions</font>

In [4]:
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.0f%s' % (round(num), ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [5]:
def get_mortgage_amoritization(
    interest,
    years,
    payments_year,
    mortgage,
    start_date,
    extra_payment=None,
    start_rent=None,
    start_cash_flow=None,
    rent_increase_yoy=None,
    extra_payment_prct=None
  ):
  # get initial values
  initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
  initial_ipmt = -1 * npf.ipmt(interest/payments_year, 1, years*payments_year, mortgage)
  initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1, years*payments_year, mortgage)

  # create dataframe
  rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
  rng.name = "Payment Date"
  initial_cols_list = ['Org Total Payment','Total Payment','Interest','Principal',
                  'Rent','Cash Flow','Additional Payment','Org Ending Balance','Ending Balance']
  df = pd.DataFrame(index=rng, columns=initial_cols_list, dtype='float')
  df.reset_index(inplace=True)
  df.index += 1
  df.index.name = "Period"

  # test if additional cash flow from rent is passed as an arg
  if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
    initial_additional_pmt = start_cash_flow * extra_payment_prct
  elif extra_payment != None:
    initial_additional_pmt = extra_payment
  else:
    initial_additional_pmt = 0


  # create first row
  period = 1
  initial_row_dict = {
    'Org Total Payment': initial_pmt,
    'Total Payment': initial_pmt + (initial_additional_pmt),
    'Interest': initial_ipmt,
    'Principal': initial_ppmt,
    'Rent': start_rent,
    'Cash Flow': start_cash_flow,
    'Additional Payment': initial_additional_pmt,
    'Org Ending Balance': mortgage - initial_ppmt,
    'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)
  }
  columns = list(initial_row_dict.keys())
  period_values = list(initial_row_dict.values())
  df.at[period, columns]=period_values
  df = df.round(2)

  # add additional rows
  for period in range(2, len(df) + 1):
    # get prior period values
    previous_total_payment = df.loc[period - 1, 'Total Payment']
    previous_principal = df.loc[period - 1, 'Principal']
    previous_rent = df.loc[period - 1, 'Rent']
    previous_cf = df.loc[period - 1, 'Cash Flow']
    previous_org_ending_balance = df.loc[period - 1, 'Org Ending Balance']
    previous_ending_balance = df.loc[period - 1, 'Ending Balance']
    
    # get additional payment values
    if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
      if period % 13 == 0:
        period_rent = previous_rent * (1 + rent_increase_yoy)
      else:
        period_rent = previous_rent
      period_cash_flow = previous_cf + (period_rent - previous_rent)
      period_additional_pmt = period_cash_flow * extra_payment_prct
    elif extra_payment != None:
      period_additional_pmt = initial_additional_pmt
      period_rent = 0
      period_cash_flow = 0
      extra_payment_prct = 0
    else:
      period_additional_pmt = 0
      period_rent = 0
      period_cash_flow = 0
      extra_payment_prct = 0

    # get end balance
    period_interest = previous_org_ending_balance * interest / payments_year
    period_principal = initial_pmt - period_interest
    org_ending_balance = previous_org_ending_balance - period_principal
    ending_balance = previous_ending_balance - period_principal - period_additional_pmt
    org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
    ending_balance = 0 if ending_balance <= 0 else ending_balance


    row_dict = {'Org Total Payment': initial_pmt,
                'Total Payment': initial_pmt + (period_cash_flow * extra_payment_prct),
                'Interest': period_interest,
                'Principal': period_principal,
                'Rent': period_rent,
                'Cash Flow': period_cash_flow,
                'Additional Payment': period_additional_pmt,
                'Org Ending Balance': org_ending_balance,
                'Ending Balance': ending_balance}
    columns = list(row_dict.keys())
    period_values = list(row_dict.values())
    df.at[period,columns]=period_values

  df = df.round(2)
  
  return df

In [6]:
def plot_mortgage_amoritization(df):
  # melt dataframe to plot
  df_plot = pd.melt(
      df, 
      id_vars=['Payment Date'], 
      value_vars=['Org Ending Balance', 'Ending Balance']
  )

  # calculate variables for chart context
  early_payment_date = df_plot.loc[(df_plot['variable'] == 'Ending Balance') & (df_plot['value'] == 0)]\
    ['Payment Date'].min().strftime('%Y-%m-%d')
  max_payment_date = df_plot['Payment Date'].max().strftime('%Y-%m-%d')
  additional_interest = df.loc[df['Payment Date'] > early_payment_date]['Interest'].sum()

  # create fig
  fig = px.line(df_plot, x='Payment Date', y='value', color='variable')
  fig.add_vline(x=early_payment_date, line_dash="dot", line_color="black")
  fig.add_vrect(
    x0=early_payment_date, 
    x1=max_payment_date, 
    fillcolor="red", 
    opacity=0.25, 
    annotation_position="top left",
    annotation_text="+${0} interest savings".format(human_format(additional_interest))
  )
  fig.update_layout(
    title='Mortgage Amoritization', 
    xaxis_title='Year', 
    yaxis_title='Mortgage Balance', 
    plot_bgcolor='white',
    legend=dict(
      title_text="Balance Type",
      yanchor="bottom",
      y=0.02,
      xanchor="left",
      x=0.01)
  )
  return fig

## <font color="blue">Locals & Constants</font>

In [7]:
interest = 0.04
years = 30
payments_year = 12
mortgage = 400000
start_date = (date(2022, 5, 1))

# SCENARIO #1
# General: Same payment each month until loan is paid off
extra_payment = 300

# SCENARIO #2
# Investor/Landlord: Dynamic payment based on % of rental cash flow
start_rent = None
start_cash_flow = None
rent_increase_yoy = None
extra_payment_prct = None

## <font color="blue">Data</font>

## <font color="blue">Transformations</font>

In [8]:
initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
initial_ipmt = -1 * npf.ipmt(interest/payments_year, 1, years*payments_year, mortgage)
initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1, years*payments_year, mortgage)
print('Initial Payment: {:,.2f}'.format(initial_pmt))
print('Initial Interest: {:,.2f}'.format(initial_ipmt))
print('Initial Principal Payment: {:,.2f}'.format(initial_ppmt))

Initial Payment: 1,909.66
Initial Interest: 1,333.33
Initial Principal Payment: 576.33


In [9]:
# create a range of dates in a pandas dataframe
rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
# label the date column
rng.name = "Payment Date"
# create dataframe 
df = pd.DataFrame(
    index=rng, 
    columns=['Org Total Payment',
            'Total Payment',
            'Interest',
            'Principal',
            'Rent',
            'Cash Flow',
            'Additional Payment',
            'Org Ending Balance',
            'Ending Balance'], dtype='float')
# set index as payment period
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"

In [10]:
df

,Payment Date,Org Total Payment,Total Payment,Interest,Principal,Rent,Cash Flow,Additional Payment,Org Ending Balance,Ending Balance
Period,,,,,,,,,,
1,2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
356,2051-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,2052-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2052-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# test if additional cash flow from rent is passed as an arg
if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
  initial_additional_pmt = start_cash_flow * extra_payment_prct
elif extra_payment != None:
  initial_additional_pmt = extra_payment
else:
  initial_additional_pmt = 0

In [14]:
# create values for first period (first row)
period = 1
# for each element in the row set the value
initial_row_dict = {
  'Org Total Payment': initial_pmt,
  'Total Payment': initial_pmt + (initial_additional_pmt),
  'Interest': initial_ipmt,
  'Principal': initial_ppmt,
  'Rent': start_rent,
  'Cash Flow': start_cash_flow,
  'Additional Payment': initial_additional_pmt,
  'Org Ending Balance': mortgage - initial_ppmt,
  'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)
}
# set values
columns = list(initial_row_dict.keys())
period_values = list(initial_row_dict.values())
df.at[period, columns]=period_values
# round values
df = df.round(2)
df

InvalidIndexError: You can only assign a scalar value not a <class 'list'>

In [15]:
# add additional rows
for period in range(2, len(df) + 1):
  # get prior period values
  previous_total_payment = df.loc[period - 1, 'Total Payment']
  previous_principal = df.loc[period - 1, 'Principal']
  previous_rent = df.loc[period - 1, 'Rent']
  previous_cf = df.loc[period - 1, 'Cash Flow']
  previous_org_ending_balance = df.loc[period - 1, 'Org Ending Balance']
  previous_ending_balance = df.loc[period - 1, 'Ending Balance']

  # get additional payment values
  if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
    if period % 13 == 0:
      period_rent = previous_rent * (1 + rent_increase_yoy)
    else:
      period_rent = previous_rent
    period_cash_flow = previous_cf + (period_rent - previous_rent)
    period_additional_pmt = period_cash_flow * extra_payment_prct
  elif extra_payment != None:
    period_additional_pmt = initial_additional_pmt
    period_rent = 0
    period_cash_flow = 0
    extra_payment_prct = 0
  else:
    period_additional_pmt = 0
    period_rent = 0
    period_cash_flow = 0
    extra_payment_prct = 0

  # get end balance
  period_interest = previous_org_ending_balance * interest / payments_year
  period_principal = initial_pmt - period_interest
  org_ending_balance = previous_org_ending_balance - period_principal
  ending_balance = previous_ending_balance - period_principal - period_additional_pmt
  org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
  ending_balance = 0 if ending_balance <= 0 else ending_balance


  row_dict = {'Org Total Payment': initial_pmt,
              'Total Payment': initial_pmt + period_additional_pmt,
              'Interest': period_interest,
              'Principal': period_principal,
              'Rent': period_rent,
              'Cash Flow': period_cash_flow,
              'Additional Payment': period_additional_pmt,
              'Org Ending Balance': org_ending_balance,
              'Ending Balance': ending_balance}
  columns = list(row_dict.keys())
  period_values = list(row_dict.values())
  df.at[period,columns]=period_values

df = df.round(2)
df

InvalidIndexError: You can only assign a scalar value not a <class 'list'>

In [16]:
df_plot = pd.melt(df, id_vars=['Payment Date'], value_vars=['Org Ending Balance', 'Ending Balance'])
df_plot

,Payment Date,variable,value
0,2022-05-01,Org Ending Balance,NaN
1,2022-06-01,Org Ending Balance,NaN
2,2022-07-01,Org Ending Balance,NaN
3,2022-08-01,Org Ending Balance,NaN
4,2022-09-01,Org Ending Balance,NaN
...,...,...,...
715,2051-12-01,Ending Balance,NaN
716,2052-01-01,Ending Balance,NaN
717,2052-02-01,Ending Balance,NaN
718,2052-03-01,Ending Balance,NaN


In [17]:
# get last payment for early payoff
early_payment_date = df_plot.loc[(df_plot['variable'] == 'Ending Balance') & (df_plot['value'] == 0)]['Payment Date'].min().strftime('%Y-%m-%d')
# get max payment date
max_payment_date = df_plot['Payment Date'].max().strftime('%Y-%m-%d')
# get savings in interest
additional_interest = df.loc[df['Payment Date'] > early_payment_date]['Interest'].sum()

print('Early payment date:', early_payment_date)
print('End payment date:', max_payment_date)
print('Additonal interest:', additional_interest)

ValueError: NaTType does not support strftime

In [18]:
# create plotly chart
fig = px.line(df_plot, x='Payment Date', y='value', color='variable')
fig.add_vline(x=early_payment_date, line_dash="dot", line_color="black")
fig.add_vrect(
  x0=early_payment_date, 
  x1=max_payment_date, 
  fillcolor="red", 
  opacity=0.25, 
  annotation_position="top left",
  annotation_text="+${0} interest savings".format(human_format(additional_interest))
)
fig.update_layout(
  title='Mortgage Amoritization', 
  xaxis_title='Year', 
  yaxis_title='Mortgage Balance', 
  plot_bgcolor='white',
  legend=dict(
    title_text="Balance Type",
    yanchor="bottom",
    y=0.02,
    xanchor="left",
    x=0.01)
)
fig.show()

NameError: name 'early_payment_date' is not defined

## <font color="blue">Call Functions</font>

In [19]:
interest = 0.065
years = 40
payments_year = 12
mortgage = 400000
start_date = (date(2022, 5, 1))

# SCENARIO #1
# General: Same payment each month until loan is paid off
extra_payment = None

# SCENARIO #2
# Investor/Landlord: Dynamic payment based on % of rental cash flow
start_rent = 2000
start_cash_flow = 500
rent_increase_yoy = .03
extra_payment_prct = 0.25

# call function to get amoritization schedule
_df = get_mortgage_amoritization(
    interest, years, payments_year, mortgage,
    start_date, extra_payment, start_rent, start_cash_flow,
    rent_increase_yoy, extra_payment_prct
)
_df.head()

InvalidIndexError: You can only assign a scalar value not a <class 'list'>

In [20]:
_df[['Rent', 'Cash Flow', 'Additional Payment']].drop_duplicates()

NameError: name '_df' is not defined

In [21]:
# call function to plot mortgage amoritization
plot_mortgage_amoritization(_df)

NameError: name '_df' is not defined

# End Notebook